In [1]:
import requests

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm_notebook


In [2]:
# Vivino 탐색을 위한 driver 생성
driver = webdriver.Chrome()
url = 'https://www.vivino.com/explore?e=eJzLLbI1VMvNzLM1UMtNrLA1NQABteRKW-8gtWQgEa5WAFSQnmZblliUmVqSmKOWX5Rim5JanKyWn1RpW5RYkpmXXhyfWJZalJieqlZeEh1rawgAgG8ccg%3D%3D'
driver.get(url)
time.sleep(3)

In [3]:
# 페이지 끝까지 스크롤하여 모든 데이터 로드
while True:
    # 현재 높이 저장
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    # 페이지 끝까지 스크롤
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # 페이지 로드 대기
    time.sleep(1)
    
    # 페이지 살짝 위로 스크롤하여 데이터 로드될 수 있도록 조정함
    driver.execute_script("window.scrollBy(0, -100);")
    time.sleep(3)
    
    #새로운 높이 계산
    new_height = driver.execute_script("return document.body.scrollHeight")

    if new_height == last_height:
        break

    last_height = new_height

In [4]:
# 반복문 종료 후 스크롤 맨 위로 올리기
driver.execute_script("window.scrollTo(0,0);")

In [5]:
# 페이지 소스 가져오기
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

# 와인 카드 데이터 가져오기 및 개수 확인
wines = soup.find_all('div', class_='wineCard__wineCard--2dj2T')
len(wines)

1961

In [ ]:
# wine 데이터 추출하기

data1 = []
base_url = 'https://www.vivino.com'

for wine in tqdm_notebook(wines):
    
    country = wine.find('div', class_="wineInfoLocation__regionAndCountry--1nEJz").get_text().split(",")[-1].strip()
    region = wine.find('div', class_="wineInfoLocation__regionAndCountry--1nEJz").get_text().split(",")[0].strip()
    rating = wine.find('div', class_="vivinoRating_averageValue__uDdPM").get_text().strip()
    nratings = wine.find('div', class_="vivinoRating_caption__xL84P").get_text().split()[0].strip() # rating 수 
    label = wine.find('div', class_="wineInfoVintage__vintage--VvWlU wineInfoVintage__truncate--3QAtw").get_text()
    winery = wine.find('div', class_ = "wineInfoVintage__truncate--3QAtw").get_text()
    price = wine.find('div', class_="addToCart__subText--1pvFt addToCart__ppcPrice--ydrd5").get_text().split()[-1].replace('₩', '')

    # Crawling wine varieties
    wine_link = wine.find('a',class_="anchor_anchor__m8Qi- wineCard__cardLink--3F_uB")['href']
    link = base_url + wine_link

    wait = WebDriverWait(driver, 10)
    facts_section = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'anchor_anchor__m8Qi-')))

    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') # 맨 밑으로 스크롤하여 모든 페이지소스 로딩
    facts_section = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'anchor_anchor__m8Qi-'))) # 페이지 로딩될까지 Wait

    driver.execute_script('window.scrollTo(0, document.body.scrollHeight);') # 맨 밑으로 스크롤하여 모든 페이지소스 로딩
    facts_section = wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'anchor_anchor__m8Qi-'))) # 페이지 로딩될까지 Wait
    
    wine_html = driver.page_source
    wine_soup = BeautifulSoup(wine_html, 'html.parser')

    grape_links= wine_soup.find_all('a', class_='anchor_anchor__m8Qi-', href = True)
    grapes = [grape.text for grape in grape_links if '/grapes/' in grape['href']]
    
    # data에 값 저장하기
    data1.append({
        'Country': country,
        'Region': region,
        'Rating': rating,
        'Nratings': nratings,
        'Label': label,
        'Winery':winery,
        'Price': price,
        'Wine_Variety': grapes
    })

df1 = pd.DataFrame(data1)
df1.to_csv('./Wine Data/vivino_wine_data.csv', index = False)


In [64]:
driver.quit()

In [2]:
import pandas as pd

df_wine = pd.read_csv('./Wine Data/vivino_wine_data.csv')
df_wine

,Unnamed: 0,index,Country,Region,Rating,Nratings,Label,Winery,Price,Wine_Variety
0,0,0,France,Côte-Rôtie,4.8,39,Les Grandes Places Côte-Rôtie 1999,Jean-Michel Gerin,"479,023",['Shiraz/Syrah']
1,1,1,France,Pauillac,4.7,3745,Pauillac (Grand Cru Classé) 2009,Château Pontet-Canet,"386,706","['Cabernet Sauvignon', 'Merlot', 'Cabernet Fra..."
2,2,2,Chile,Apalta Valley,4.7,1101,Clos Apalta 2005,Clos Apalta,"127,752","['Carménère', 'Cabernet Sauvignon', 'Merlot', ..."
3,3,3,France,Saint-Émilion Grand Cru,4.7,120,Saint-Émilion Grand Cru (Grand Cru Classé) 2005,Château Pavie Decesse,"298,615","['Cabernet Franc', 'Merlot']"
4,4,4,Chile,Aconcagua Valley,4.6,1516,Aconcagua Valley 2007,Sena,"172,465",['Cabernet Franc']
...,...,...,...,...,...,...,...,...,...,...
1956,1956,1956,Chile,Maule Valley,2.8,128,Merlot 2010,Aromo,"23,421",['Merlot']
1957,1957,1957,Spain,Cariñena,2.8,57,Xmas Special Edition Cariñena 2012,Beso de Vino,"23,847",['Cariñena']
1958,1958,1958,Chile,Central Valley,2.7,61,Casa Lo Matta Cabernet Sauvignon 2010,Aromo,"19,163",['Cabernet Sauvignon']
1959,1959,1959,Chile,Central Valley,2.7,28,Cabernet - Carmenère 2006,El Emperador,"22,144",['Carménère']


In [27]:
def return_first_value(wine_variety):
    if len(wine_variety) == 0:
        return None
    return wine_variety[0]

In [15]:
wine_list = df_wine.loc[1, 'Wine_Variety']
wine_list

"['Cabernet Sauvignon', 'Merlot', 'Cabernet Franc', 'Petit Verdot']"

In [25]:
import re

def str_to_list(x):
    if x == "[]":
        return []
    return re.findall(r"'(.*?)'", x)


df_wine['Wine_Variety'] = df_wine['Wine_Variety'].apply(str_to_list)

In [29]:
df_wine['Main_Variety'] = df_wine['Wine_Variety'].apply(return_first_value)
df_wine

,Unnamed: 0,index,Country,Region,Rating,Nratings,Label,Winery,Price,Wine_Variety,Main_Variety,Main Variety
0,0,0,France,Côte-Rôtie,4.8,39,Les Grandes Places Côte-Rôtie 1999,Jean-Michel Gerin,"479,023",[Shiraz/Syrah],Shiraz/Syrah,Shiraz/Syrah
1,1,1,France,Pauillac,4.7,3745,Pauillac (Grand Cru Classé) 2009,Château Pontet-Canet,"386,706","[Cabernet Sauvignon, Merlot, Cabernet Franc, P...",Cabernet Sauvignon,Cabernet Sauvignon
2,2,2,Chile,Apalta Valley,4.7,1101,Clos Apalta 2005,Clos Apalta,"127,752","[Carménère, Cabernet Sauvignon, Merlot, Petit ...",Carménère,Carménère
3,3,3,France,Saint-Émilion Grand Cru,4.7,120,Saint-Émilion Grand Cru (Grand Cru Classé) 2005,Château Pavie Decesse,"298,615","[Cabernet Franc, Merlot]",Cabernet Franc,Cabernet Franc
4,4,4,Chile,Aconcagua Valley,4.6,1516,Aconcagua Valley 2007,Sena,"172,465",[Cabernet Franc],Cabernet Franc,Cabernet Franc
...,...,...,...,...,...,...,...,...,...,...,...,...
1956,1956,1956,Chile,Maule Valley,2.8,128,Merlot 2010,Aromo,"23,421",[Merlot],Merlot,Merlot
1957,1957,1957,Spain,Cariñena,2.8,57,Xmas Special Edition Cariñena 2012,Beso de Vino,"23,847",[Cariñena],Cariñena,Cariñena
1958,1958,1958,Chile,Central Valley,2.7,61,Casa Lo Matta Cabernet Sauvignon 2010,Aromo,"19,163",[Cabernet Sauvignon],Cabernet Sauvignon,Cabernet Sauvignon
1959,1959,1959,Chile,Central Valley,2.7,28,Cabernet - Carmenère 2006,El Emperador,"22,144",[Carménère],Carménère,Carménère


In [32]:
df_wine.to_csv('./Wine Data/vivino_wine_data.csv')